In [21]:
import notebookjs
from notebookjs import execute_js
import pandas as pd
import numpy as np
import sys

# sys.path.insert(0, '../core/')
# from pycalibrate.calibrate import Calibrate
from pysure import sure

In [2]:
import json

file = "data/loan/test.json"
with open(file, 'r') as json_input:
    data = json.load(json_input)
attrs = data['columns']
df = pd.DataFrame(data=np.array(data['data']), columns = attrs)
y_pred = data['y_pred']
y_gt = data['y_gt']

In [3]:
import importlib
from surrogate_rule import forest_info
from surrogate_rule import tree_node_info
importlib.reload(tree_node_info)
importlib.reload(forest_info)

<module 'surrogate_rule.forest_info' from '/Users/junyuan/Documents/_/github/research/pysure/surrogate_rule/forest_info.py'>

In [4]:
filter_threshold = {
    "support": 5,
    "fidelity": .85,
    "num_feat": 4,
    "num_bin": 3,
}

## Descriptive Rules

In [5]:
surrogate_obj = tree_node_info.tree_node_info()

surrogate_obj.initialize(X=np.array(data['data']), y=np.array(data['y_gt']), 
                         y_pred=np.array(data['y_pred']), debug_class=-1,
                         attrs=data['columns'], filter_threshold=filter_threshold,
                         n_cls=len(data['target_names']),
                         num_bin=filter_threshold['num_bin'], verbose=False
).train_surrogate_random_forest().tree_pruning()

forest_obj = tree_node_info.forest()
forest_obj.initialize(
    trees=surrogate_obj.tree_list, cate_X=surrogate_obj.cate_X, 
    y=surrogate_obj.y, y_pred=surrogate_obj.y_pred, attrs=attrs, 
    num_bin=filter_threshold['num_bin'],
    real_percentiles=surrogate_obj.real_percentiles,
    real_min=surrogate_obj.real_min, real_max=surrogate_obj.real_max,
).construct_tree().extract_rules()

forest = forest_info.Forest()

forest.initialize(forest_obj.tree_node_dict, data['real_min'], data['real_max'], surrogate_obj.percentile_info,
    df, data['y_pred'], data['y_gt'],
    forest_obj.rule_lists,
    data['target_names'], 2)
forest.initialize_rule_match_table()
forest.initilized_rule_overlapping()

res = forest.find_the_min_set()

## Rules for Errors

In [7]:
from surrogate_rule import forest_info
from surrogate_rule import tree_node_info
importlib.reload(tree_node_info)
importlib.reload(forest_info)


surrogate_obj = tree_node_info.tree_node_info()

surrogate_obj.initialize(X=np.array(data['data']), y=np.array(data['y_gt']), 
                         y_pred=np.array(data['y_pred']), debug_class=-1,
                         attrs=data['columns'], filter_threshold=filter_threshold,
                         n_cls=len(data['target_names']),
                         num_bin=filter_threshold['num_bin'], 
                         error_analysis = True,
                         verbose=True
).train_surrogate_random_forest().tree_pruning()

forest_obj = tree_node_info.forest()
forest_obj.initialize(
    trees=surrogate_obj.tree_list, cate_X=surrogate_obj.cate_X, 
    y=surrogate_obj.y, y_pred=surrogate_obj.y_pred, attrs=attrs, 
    num_bin=filter_threshold['num_bin'],
    real_percentiles=surrogate_obj.real_percentiles,
    real_min=surrogate_obj.real_min, real_max=surrogate_obj.real_max,
).construct_tree().extract_rules()

forest = forest_info.Forest()

forest.initialize(forest_obj.tree_node_dict, data['real_min'], data['real_max'], 
                  surrogate_obj.percentile_info,
                  df, data['y_pred'], data['y_gt'],
                  forest_obj.rule_lists,
                  data['target_names'], 2,
                  error_analysis = True,
                 )
forest.initialize_rule_match_table()
forest.initilized_rule_overlapping()

res = forest.find_the_min_set()

***** finish transforming ordinal data *****
***** finish training surrogate random forest *****
surrogate overall fidelity: 0.9958841222099097
[0.06377149 0.06496322 0.0645268  0.05282231 0.         0.
 0.02816479 0.03189897 0.01138862 0.03212378 0.05543128 0.05505222
 0.07204397 0.04401116 0.0240357  0.02360837 0.05753603 0.08208565
 0.06114538 0.07615238 0.03498352 0.06425433]
***** finish node indexing for tree no. 0 / 100 *****
***** finish node indexing for tree no. 1 / 100 *****
***** finish node indexing for tree no. 2 / 100 *****
***** finish node indexing for tree no. 3 / 100 *****
***** finish node indexing for tree no. 4 / 100 *****
***** finish node indexing for tree no. 5 / 100 *****
***** finish node indexing for tree no. 6 / 100 *****
***** finish node indexing for tree no. 7 / 100 *****
***** finish node indexing for tree no. 8 / 100 *****
***** finish node indexing for tree no. 9 / 100 *****
***** finish node indexing for tree no. 10 / 100 *****
***** finish node inde

In [8]:
res['text_rules']

['If Months Since Last Delq.>=150.0 AND Unpaid Balance<47.0 Then 33.0% error (2133/6317)',
 'If Average Months in File>=64.0 AND Bills Paid on Time>=100.0 Then 30.0% error (1896/6317)',
 'If Months Since Oldest Trade Open>=154.0 AND Unpaid Balance<14.0 Then 23.0% error (1488/6317)',
 'If Satisfactory Trades>=15.0 AND Unpaid Balance<14.0 Then 21.0% error (1388/6317)',
 'If Months Since Oldest Trade Open>=230.0 AND Inq Last 6 Months exl. 7 days<2.0 AND Unpaid Balance<47.0 Then 17.0% error (1130/6317)',
 'If Average Months in File>=64.0 AND Months Since Last Delq.>=23.0 AND Unpaid Balance<47.0 AND % Trades w/ Balance>=56.0 Then 16.0% error (1021/6317)',
 'If Total Trades>=26.0 AND Months Since Most Recent Inq<6.0 AND Unpaid Balance<47.0 AND % Trades w/ Balance<56.0 Then 8.0% error (537/6317)',
 'If Months Since Last Delq. from 23.0 to 150.0 AND Max Delq. Ever>=6.0 AND % Trades w/ Balance<56.0 Then 3.0% error (248/6317)']

## Visual Explroation of Rules

In [28]:
importlib.reload(sure)

## initialize SuRE with data and default parameters 
s = sure.SuRE(data=data, error_analysis=False)

## plotting
s.visualize()